In [1]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import datetime
import numpy as np
#import tensorflow as tf

#  Import and read the file.
supply_df = pd.read_csv("./CAISO-ALL-SUPPLY-DATA.csv")
supply_df.head()

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime
0,50036.0,5568.0,78.0,2353.0,-447.0,2265.0,21.0,0.0,7/5/21,11:50:00,7/5/21 11:50
1,8695.0,26540.0,5081.0,4575.0,119.0,2248.0,21.0,0.0,8/18/20,16:25:00,8/18/20 16:25
2,9301.0,26159.0,4408.0,5035.0,67.0,2250.0,23.0,0.0,8/18/20,15:55:00,8/18/20 15:55
3,9343.0,26132.0,4382.0,5032.0,49.0,2250.0,22.0,0.0,8/18/20,15:50:00,8/18/20 15:50
4,9283.0,26015.0,4300.0,5278.0,63.0,2248.0,22.0,0.0,8/18/20,15:40:00,8/18/20 15:40


In [2]:
supply_df[supply_df['Batteries'] < 0] = 0
supply_df[supply_df['Nuclear'] < 0] = 0
supply_df[supply_df['Coal'] < 0] = 0
supply_df[supply_df['Other'] < 0] = 0


In [3]:
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Large hydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Natural gas']



In [4]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

          0    1    2
0       NaN  NaN  NaN
1        16   25   00
2        15   55   00
3        15   50   00
4        15   40   00
...     ...  ...  ...
412947    2   35   00
412948    2   40   00
412949    2   45   00
412950    2   50   00
412951    2   55   00

[412952 rows x 3 columns]


In [5]:
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)

ValueError: cannot convert float NaN to integer

In [ ]:
baby_df= baby_df.drop([2],1)

In [ ]:
# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})

In [ ]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year

#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)



baby_df.head(100)

In [ ]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [ ]:
#Create a date attribute for groupby function 
supply_df['Month-D.A'] = supply_df['Date'].dt.month

supply_df['Day-DA'] = supply_df['Date'].dt.day
supply_df['Month-D.A'] = supply_df['Month-D.A'].astype(str)
supply_df['Day-DA'] = supply_df['Day-DA'].astype(str)

supply_df['DateAttribute'] = supply_df['Month-D.A'] + supply_df['Day-DA']


supply_df

In [ ]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

In [ ]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [ ]:
supply_df['Date'] = supply_df['Date'].apply(fun)
supply_df.head(1300)

In [ ]:
final_df = pd.merge(baby_df, supply_df, left_index=True, right_index=True, how='outer')

In [ ]:
final_df.head()

In [ ]:
final_df= final_df.drop(['Renewables', 'Batteries', 'Large hydro', 'Imports', 'Other', 'Nuclear', 'Coal', 'DateTime'],1)
final_df.head(1000)

In [ ]:
supply_df.dtypes

In [ ]:
final_df= final_df.drop(['Time'],1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
axes = final_df.plot.line(subplots=True)

In [ ]:
import seaborn as sns
sns.pairplot(data=final_df, diag_kind='kde')

In [ ]:
sns.heatmap(final_df[['Natural gas','Sum', 'Date']].corr(), cmap='Blues', annot=True)
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median', missing_values=np.nan)
imputer = imputer.fit(final_df[['Natural gas','Sum']])
final_df[['Natural gas','Sum']] = imputer.transform(final_df[['Natural gas','Sum']])
final_df

In [ ]:
final_df= final_df.drop(['Year', 'DemandMW', 'Month-D.A', 'Day-DA', 'DateAttribute'],1)

In [ ]:
# Create our features
X = final_df.drop("Natural gas", axis = 1)

# Create our target
y = final_df.loc[:, 'Natural gas'].copy()

In [ ]:
final_df.isnull().values.any()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)
print(model.predict(X_train))


In [ ]:
print(training_score)

In [ ]:
print(testing_score)

In [ ]:
plt.scatter(model.predict(X_train), model.predict(X_train) - y_train, c="red", label="Training Data")
plt.scatter(model.predict(X_test), model.predict(X_test) - y_test, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=6000, xmax=1500)
plt.title("Residual Plot")
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X, y)
score = model.score(X, y)
print(f"R2 Score: {score}")

Predicting Future Natural Gas Demand

In [ ]:
df2030 = pd.read_csv("./CAISO-ALL-SUPPLY-DATA.csv")
df2030.head()

In [ ]:
df2030[df2030['Renewables'] < 0] = 0
df2030[df2030['Natural gas'] < 0] = 0
df2030[df2030['Large hydro'] < 0] = 0
df2030[df2030['Imports'] < 0] = 0
df2030[df2030['Batteries'] < 0] = 0
df2030[df2030['Nuclear'] < 0] = 0
df2030[df2030['Coal'] < 0] = 0
df2030[df2030['Other'] < 0] = 0


In [ ]:
df2030['Sum'] = df2030['Renewables'] + df2030['Large hydro'] + df2030['Imports'] + df2030['Batteries'] + df2030['Nuclear'] + df2030['Coal'] + df2030['Other'] + df2030['Natural gas']

In [ ]:
other_df = df2030['Time'].str.split(":", expand=True)
print(other_df)

In [ ]:
#Format the baby_df into int.
other_df[0] = other_df[0].astype(int)
other_df[1] = other_df[1].astype(int)
#Clean baby_df
other_df= other_df.drop([2],1)

# Rename columns and create years value for calculating demand_EV
other_df = other_df.rename(columns={other_df.columns[0]: 'Hours'})
other_df = other_df.rename(columns={other_df.columns[1]: 'Minutes'})
other_df

In [ ]:
df2030['Date'] = pd.to_datetime(df2030['Date'])

#Create a new column for'Year'
other_df['Year'] = df2030['Date'].dt.year
#Convert to integers to be used in computations
other_df['Year'] = other_df['Year'].astype(int)

other_df.head(100)

In [ ]:
#Make a date string column 
date_df = pd.DataFrame()
df2030['Date1']=df2030['Date'].astype(str)

date_df = df2030['Date1'].str.split("-", expand=True)
date_df['DateAttribute'] = date_df[1] + date_df[2]
date_df = date_df.drop([0,1,2],1)

In [ ]:
calc_df = pd.merge(other_df, date_df, left_index=True, right_index=True, how='outer')
calc_df.head(-5)

In [ ]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [ ]:
df2030['Date'] = df2030['Date'].apply(fun)
df2030.head(1300)

In [ ]:
df2030 = df2030.drop(['Date1'],1)
df2030final = pd.merge(df2030, calc_df, left_index=True, right_index=True, how='outer')
df2030final
df2030final['DateTime'] = pd.to_datetime(df2030final['DateTime'])

In [ ]:
df1= pd.DataFrame()
df1 = df2030final.groupby(['DateAttribute','Time'])['Sum'].agg('mean')
df1 = df1.reset_index()
df1 = df1.rename(columns={df1.columns[2]: 'AvgSUM(3yr)'}).round(2)
df2 = df1.copy

print(len(df1))
df1.dtypes

In [ ]:
df1['Date1']= "2020" + df1['DateAttribute']
df1['pDateTime'] = df1['Date1'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['pDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
ev_df2=df1.drop(['DateAttribute','Date1','pDateTime','Time'],1)

In [ ]:
df1 = df1.drop(df1.index[16992:17280])
df1 = df1.reset_index(drop=True)
print(len(df1))
df1

In [ ]:
df1= df1.drop(['DateAttribute', 'Date1', 'pDateTime'],1)
df1.head(1000)

In [ ]:
time_df = df1['Time'].str.split(":", expand=True)
time_df.head()

In [ ]:
#Format the baby_df into int.
time_df[0] = time_df[0].astype(int)
time_df[1] = time_df[1].astype(int)


# Rename columns and create years value for calculating demand_EV
time_df = time_df.rename(columns={time_df.columns[0]: 'Hours'})
time_df = time_df.rename(columns={time_df.columns[1]: 'Minutes'})


In [ ]:
final_demand_df = pd.merge(time_df, df1, left_index=True, right_index=True, how='outer')
final_demand_df.head()

In [ ]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [ ]:
final_demand_df['Date2'] = final_demand_df['Date2'].apply(fun)

In [ ]:
final_demand_df= final_demand_df.drop(['Time', 2],1)
df1.head(1000)

In [ ]:
final_demand_df.head(1000)

In [ ]:
final_demand_df = final_demand_df.rename(columns={'Date2': 'Day', 'AvgSUM(3yr)': 'AvgSum'})

In [ ]:
final_demand_df.head()

In [ ]:
len(final_demand_df)

In [ ]:
mask = (final_demand_df['Hours'] < 7) | (final_demand_df['Hours'] > 21)
final_demand_df.loc[mask, 'AvgSum'] += 106.68

In [ ]:
final_demand_df.head(5000)

In [ ]:
print(len(final_demand_df.loc[final_demand_df['AvgSum'] < 0]))
print(final_demand_df['AvgSum'].mean())
print(final_df['Sum'].mean())


In [ ]:
final_demand_df.head()

In [ ]:
final_demand_df['Predicted 2030 NatGas'] = model.predict(final_demand_df)

In [ ]:
len(final_demand_df)

In [ ]:
df2030.head()

In [ ]:
l = final_df["Natural gas"].median()
p = final_demand_df["Predicted 2030 NatGas"].median()
print((((p/l)-1))*100)

In [ ]:
print(p-l)

In [ ]:
(-1033.4518453045357/l)*100

In [ ]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000
# gca stands for 'get current axis'
ax = plt.gca()

final_df.plot(kind='line',x='Date',y='Natural gas',ax=ax)
final_demand_df.plot(kind='line',x='Day',y='Predicted 2030 NatGas', color='red', ax=ax)
    
    # adjust the y axis scale.
#ax.locator_params('y', nbins=5)
    # adjust the x axis scale.
#plt.locator_params('x', nbins=20)
    # show the final figure.
plt.show()  
